<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🔀 תהליך הסבת טבלת סדר פעולות לבנק"ל מודרני 🔀 

In [ ]:
print('10. Sequence Actions Table:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

In [ ]:
print('  10.a Reading inputs')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [74]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

today =  dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day)
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name + '_ncdb.gdb'

<h1 style="text-align:right;font-size:100%">
טבלת סדר פעולות קיימת

In [73]:
action_parcel_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'ACTION_NUM', 'LINE', 'TALAR_ID', 'FROM_PARCEL_TMP', 'FROM_PARCEL_FINAL', 'TO_GUSH_NUM', 'TO_GUSH_SUFFIX', 'TO_PARCEL_TMP', 'TO_PARCEL_FINAL']

action_parcel = pd.read_csv(RawData_folder + '/Oracle/action_parcel.csv', usecols = action_parcel_cols)

del action_parcel_cols

<h1 style="text-align:right;font-size:100%">
טבלת עדכון חלקות קיימת

In [21]:
revision_action_cols = ['ACTION_TYPE', 'TALAR_ID', 'ACTION_NUM']

revision_action = pd.read_csv(RawData_folder + '/Oracle/REV_ACTION.csv', usecols = revision_action_cols)

del revision_action_cols

<h1 style="text-align:right;font-size:100%">
טבלת סטאטוסים של תצ"ר

In [22]:
talar_table_cols = ['TALAR_ID', 'STATUS']

talar_table = pd.read_csv(RawData_folder + '/Oracle/talar_table.csv', usecols = talar_table_cols, encoding = 'cp1256')

del talar_table_cols

<h1 style="text-align:right;font-size:100%"> 
טבלת פעולות של תצ"ר כרסטת (הוכן מראש)

In [23]:
talar_card = pd.read_csv(RawData_folder + '/Oracle/TALAR_ACTIONS.csv')

<h1 style="text-align:right;font-size:100%">
טבלת חלקות בתהליך

In [24]:
parcel_inprocess_cols = ['GUSHNUM', 'GUSHSUFFIX', 'PARCELNAME', 'TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS']

parcel_inprocess = pd.read_csv(RawData_folder + '/Oracle/PARCELS_INPROCESS.csv', usecols = parcel_inprocess_cols, encoding = 'cp1256')

del parcel_inprocess_cols

<h1 style="text-align:right;font-size:100%">
טבלת תת"ג

In [25]:
tatag_table_cols = ['TATAG_NUM', 'TATAG_YEAR', 'TATAG_STATUS']

tatag_table = pd.read_csv(RawData_folder + '/Oracle/TATAG.csv', usecols = tatag_table_cols)

del tatag_table_cols

<h1 style="text-align:right;font-size:100%">
טבלת תיקוני 97

In [26]:
tikun_hesder_97b_cols = ['MISPAR_PNIA', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCELS', 'CODE_STEP']

tikun_hesder_97b = pd.read_csv(RawData_folder + '/Oracle/TIKUN_HESDER_97B.csv', usecols = tikun_hesder_97b_cols)

del tikun_hesder_97b_cols

<h1 style="text-align:right;font-size:100%">
פסקי דין

In [27]:
judgements_cols = ['PSAK_DIN_ID', 'STATUS', 'GUSH_NUM', 'GUSH_SUFFIX']

judgements = pd.read_csv(RawData_folder + '/Oracle/psak_din.csv', usecols = judgements_cols, encoding = 'cp1255')

del judgements_cols

<h1 style="text-align:right;font-size:100%">
חלקות פסקי דין

In [28]:
judgements_parcels_cols = ['PSAK_DIN_ID', 'PARCEL', 'TYPE']

judgements_parcels = pd.read_csv(RawData_folder + '/Oracle/psak_din_parcels.csv', usecols = judgements_parcels_cols)

del judgements_parcels_cols

<h1 style="text-align:right;font-size:100%"> 
שכבת גבולות תהליכי קדסטר

In [75]:
CPB_layer = gpd.read_file(product_path, layer = 'CadasterProcessBorders', include_fields = ['CPBUniqueID', 'ProcessName', 'ProcessType'], ignore_geometry = True)
CPB_layer = CPB_layer.astype({'CPBUniqueID' : int, 'ProcessName' : str})

del [product_path]

In [ ]:
print('  10.b Cleaning data')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:100%"> 
שמות שדות לאפיון

In [76]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        ("ProcessType", int),
                        ("BlockNumber", int),
                        ("SubBlockNumber", int),
                        ("ActionNumber", int),
                        ("ActionType", int),
                        ("LineNumber", int),
                        ("FromParcelTemp", int),
                        ("FromParcelFinal", int),
                        ("ToBlockNumber", int),
                        ("ToSubBlockNumber", int),
                        ("ToParcelTemp", int),
                        ("ToParcelFinal", int)
                        ])

In [ ]:
print('  10.c Data conversion')

<h1 style="text-align:right;font-size:125%"> 
:נתוני תצ"ר
<h1 style="text-align:right;font-size:100%"> 
סינון תצ"רים בסטאטוס 3 ו-4 בלבד
<h1 style="text-align:right;font-size:100%"> 
סינון רשומות מטבלת פעולות כללית שרלוונטיים לתצ"ר
<h1 style="text-align:right;font-size:100%">
יצירת טבלה ריקה עם שדות רלונטיים
<h1 style="text-align:right;font-size:100%">
איכלוס שדות בטבלה חדשה
<h1 style="text-align:right;font-size:100%">
הגדרת כל השדות בטבלה לשדות מסוג מספר שלם

In [57]:
talar_action = pd.DataFrame(np.empty(0, dtype = new_columns))

In [ ]:
talar_action['CPBUniqueID'] = action_parcel['TALAR_ID'] + 1000000
CPB__tazar = CPB_layer[CPB_layer['ProcessType'] == 1]                 # סינון רשומות לתצ"ר 
talar_action = action_parcel[action_parcel['TALAR_ID'].isin(CPB__tazar['CPBUniqueID'])]
action_parcel['TALAR_ID'] = action_parcel['TALAR_ID'].astype('int32')

talar_action['ProcessType'] = 1


talar_action['BlockNumber'] = action_parcel['GUSH_NUM']


talar_action['SubBlockNumber'] = action_parcel['GUSH_SUFFIX']


talar_action['ActionNumber'] = action_parcel['ACTION_NUM']


talar_action['ActionType'] = action_parcel.merge(right    = revision_action,
                                                 left_on  = ['TALAR_ID', 'ACTION_NUM'],
                                                 right_on = ['TALAR_ID', 'ACTION_NUM'],
                                                 how      = 'left')['ACTION_TYPE']


talar_action['LineNumber'] = action_parcel['LINE']


talar_action['FromParcelTemp'] = action_parcel['FROM_PARCEL_TMP']


talar_action['FromParcelFinal'] = action_parcel['FROM_PARCEL_FINAL']


talar_action['ToBlockNumber'] = action_parcel['TO_GUSH_NUM']


talar_action['ToSubBlockNumber'] = action_parcel['TO_GUSH_SUFFIX']


talar_action['ToParcelTemp'] = action_parcel['TO_PARCEL_TMP']


talar_action['ToParcelFinal'] = action_parcel['TO_PARCEL_FINAL']

In [ ]:
talar_action = talar_action.astype('Int64')

<h1 style="text-align:right;font-size:125%">
נתוני תצ"ר כרטסת:
<h1 style="text-align:right;font-size:100%">
שינוי שמות שדות לפי אפיון חדש
<h1 style="text-align:right;font-size:100%">    
הגדרת סוג תהליך לתצ"ר

In [ ]:
talar_card.rename(columns = {'TALARID'             : 'CPBUniqueID',
                             'BLOCKNUMBER'         : 'BlockNumber',
                             'BLOCKSUFFIXNUMBER'   : 'SubBlockNumber',
                             'ACTIONNUMBER'        : 'ActionNumber',
                             'ACTIONTYPE'          : 'ActionType',
                             'LINENUMBER'          : 'LineNumber',
                             'FROMPARCELTEMP'      : 'FromParcelTemp',
                             'FROMPARCELFINAL'     : 'FromParcelFinal',
                             'TOBLOCKNUMBER'       : 'ToBlockNumber',
                             'TOBLOCKSUFFIXNUMBER' : 'ToSubBlockNumber',
                             'TOPARCELTEMP'        : 'ToParcelTemp',
                             'TOPARCELFINAL'       : 'ToParcelFinal'},
                  inplace = True)

talar_card['ProcessType'] = 1

<h1 style="text-align:right;font-size:125%">
:נתוני תת"ג
<h1 style="text-align:right;font-size:100%">
סינון חלקות בתהליך לחלקות שמקורן בתהליך מסוג תת"ג    
<h1 style="text-align:right;font-size:100%">
סינון טבלת תת"ג לתת"ג בסטאטוס 2    
<h1 style="text-align:right;font-size:100%">
יצירת שדה משותף בשתי הטבלאות לפי מספר מלבן   
<h1 style="text-align:right;font-size:100%">
סינון חלקות בתהליך לפי מספרי מלבן משותפים לטבלת תת"ג    
<h1 style="text-align:right;font-size:100%">
פיצול שדה שם החלקה לפי סטאטוס ומספר חלקה    
<h1 style="text-align:right;font-size:100%">
סינון רשומות שמופיעות רק בגבולות תת"ג של מוצר עדכני
<h1 style="text-align:right;font-size:100%">
מיזוג שדות שלשכבת תת"ג עדכנית עם חלקות תת"ג בתהליך    
<h1 style="text-align:right;font-size:100%">
יצירת טבלה ריקה עם שדות רלוונטיים    
<h1 style="text-align:right;font-size:100%">
איכלוס שדות בטבלה חדשה    
<h1 style="text-align:right;font-size:100%">
החלפת ערכים ריקים עם המספר 0 בשדה תת-גוש   
<h1 style="text-align:right;font-size:100%">
הגדרת כל השדות בטבלה לשדות מסוג מספר שלם    

In [ ]:
parcel_inprocess = parcel_inprocess[parcel_inprocess['CADASTER_PROCESS'] == 12]

In [ ]:
tatag_table = tatag_table[tatag_table['TATAG_STATUS'] == 2]

In [ ]:
parcel_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = parcel_inprocess[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
tatag_table[['TATAG_NUM', 'TATAG_YEAR']] = tatag_table[['TATAG_NUM', 'TATAG_YEAR']].astype(int)

parcel_inprocess['ProcessName'] = parcel_inprocess['TALAR_NUM'].astype(str) + '/' + parcel_inprocess['TALAR_YEAR'].astype(str)
tatag_table['ProcessName'] = tatag_table['TATAG_NUM'].astype(str) + '/' + tatag_table['TATAG_YEAR'].astype(str)

In [ ]:
parcel_inprocess = parcel_inprocess[parcel_inprocess['ProcessName'].isin(CPB_layer['ProcessName'])]

In [ ]:
parcel_inprocess[['ParcelStatus', 'ParcelNumber']] = parcel_inprocess['PARCELNAME'].str.extract('([a-zA-Z]+)([^a-zA-Z]+)', expand=True)
parcel_inprocess.drop(columns='PARCELNAME', inplace=True)
parcel_inprocess['ParcelNumber'] = parcel_inprocess['ParcelNumber'].fillna(0)
parcel_inprocess['ParcelNumber'] = parcel_inprocess['ParcelNumber'].astype(int)

In [ ]:
tatag_layer = CPB_layer[CPB_layer['ProcessName'].isin(parcel_inprocess['ProcessName'])]
tatag_parcel_inprocess = tatag_layer.merge(parcel_inprocess, on='ProcessName', how='right')

del [tatag_layer, parcel_inprocess, tatag_table]

In [ ]:
tatag_action = pd.DataFrame(np.empty(0, dtype=new_columns))

In [ ]:
tatag_action['CPBUniqueID'] = tatag_parcel_inprocess['CPBUniqueID']


tatag_action['ProcessType'] = 4


tatag_action['BlockNumber'] = tatag_parcel_inprocess['GUSHNUM']


tatag_action['SubBlockNumber'] = tatag_parcel_inprocess['GUSHSUFFIX']


tatag_action['ActionNumber'] =  None


tatag_action['ActionType'] = 4


tatag_action['LineNumber'] = None


tatag_action['FromParcelTemp'] =   None


tatag_action['FromParcelFinal'] = np.where(tatag_parcel_inprocess['ParcelStatus']=='F',
                                           tatag_parcel_inprocess['ParcelNumber'],
                                           None)


tatag_action['ToBlockNumber'] = None


tatag_action['ToSubBlockNumber'] = None


tatag_action['ToParcelTemp'] = np.where(tatag_parcel_inprocess['ParcelStatus']=='T',
                                        tatag_parcel_inprocess['ParcelNumber'].astype('Int64'),
                                        None)


tatag_action['ToParcelFinal'] = np.where(tatag_parcel_inprocess['ParcelStatus']=='F',
                                           tatag_parcel_inprocess['ParcelNumber'].astype('Int64'),
                                            None) # equals to 'FromParcelFinal'

In [ ]:
tatag_action[['FromParcelFinal', 'ToParcelTemp', 'ToParcelFinal']].replace(0, None, inplace=True);
tatag_action = tatag_action.astype('Int64')

<h1 style="text-align:right;font-size:125%">
:נתוני 97ב
<h1 style="text-align:right;font-size:100%"> 
סינון תיקוני 97ב' לפי סטאטוס 5 ו-6
<h1 style="text-align:right;font-size:100%">
פיצול תיקוני 97 ב' לרשומות לפי מספר חלקה    
<h1 style="text-align:right;font-size:100%">
מיון ערכים לפי מספר פניה ומספר חלקה    
<h1 style="text-align:right;font-size:100%">    
הגדרת ואיכלוס שדות

In [ ]:
tikun_hesder_97b = tikun_hesder_97b[tikun_hesder_97b['CODE_STEP'].isin([5, 6])]

In [ ]:
tikun_hesder_97b = tikun_hesder_97b.assign(PARCELS=tikun_hesder_97b.PARCELS.str.split(',')).explode('PARCELS').reset_index(drop=True)
tikun_hesder_97b.sort_values(['MISPAR_PNIA', 'PARCELS'], inplace=True)

In [ ]:
b97_action = pd.DataFrame(np.empty(0, dtype=new_columns))

In [ ]:
b97_action['CPBUniqueID'] = tikun_hesder_97b['MISPAR_PNIA'] + 5000000
b97_action = b97_action[b97_action['CPBUniqueID'].isin(CPB_layer['CPBUniqueID'])]


b97_action['ProcessType'] = 6


b97_action['BlockNumber'] = tikun_hesder_97b['GUSH_NUM'].astype(int)


b97_action['SubBlockNumber'] = tikun_hesder_97b['GUSH_SUFFIX'].astype(int)


b97_action['ActionNumber'] = None


b97_action['ActionType'] = 4


b97_action['LineNumber'] = None


b97_action['FromParcelTemp'] = None


b97_action['FromParcelFinal'] = tikun_hesder_97b['PARCELS']


b97_action['ToBlockNumber'] = None


b97_action['ToSubBlockNumber'] = None


b97_action['ToParcelTemp'] = None


b97_action['ToParcelFinal'] = tikun_hesder_97b['PARCELS']

<h1 style="text-align:right;font-size:125%">
:נתוני פסקי דין
<h1 style="text-align:right;font-size:100%">    
סינון פסקי דין לפי סטאטוס 1 ו-3
<h1 style="text-align:right;font-size:100%">
מיזוג פסקי דין עם חלקות של פסקי דין
<h1 style="text-align:right;font-size:100%">
יצירת טבלה ריקה עם שדות רלוונטיים
<h1 style="text-align:right;font-size:100%">
הגדרת ואיכלוס שדות

In [ ]:
judgements = judgements[judgements['STATUS'].isin([1,3])]

In [ ]:
judgements_merge = judgements.merge(judgements_parcels, on='PSAK_DIN_ID', how='left').astype('Int32')
judgements_merge.dropna(subset=['TYPE'], inplace=True)
judgements_merge.reset_index(inplace=True, drop=True)

In [ ]:
judgements_action = pd.DataFrame(np.empty(0, dtype=new_columns))

In [ ]:
judgements_action['FromParcelFinal'] = np.where(judgements_merge['TYPE'] == 0,
                                                judgements_merge['PARCEL'],
                                                None)


judgements_action['ToParcelFinal'] = np.where(judgements_merge['TYPE'] == 1,
                                               judgements_merge['PARCEL'],
                                               None)


judgements_action['CPBUniqueID'] = judgements_merge['PSAK_DIN_ID'] + 4000000
judgements_action = judgements_action[judgements_action['CPBUniqueID'].isin(CPB_layer['CPBUniqueID'])]


judgements_action['ProcessType'] = 3


judgements_action['BlockNumber'] = judgements_merge['GUSH_NUM']


judgements_action['SubBlockNumber'] = judgements_merge['GUSH_SUFFIX'].fillna(0)


judgements_action['ActionNumber'] = None


judgements_action['ActionType'] = 0


judgements_action['LineNumber'] = None


judgements_action['FromParcelTemp'] = None


judgements_action['ToBlockNumber'] = None


judgements_action['ToSubBlockNumber'] = None


judgements_action['ToParcelTemp'] = None

## איחוד תוצאות

In [ ]:
print('  10.d Defining new Layer')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨
<h1 style="text-align:right;font-size:100%">
איחוד של 4 טבלאות פעולה לטבלת סדר פעולות אחת

In [ ]:
SequenceActionsTable = talar_action.append(talar_card)\
                                   .append(tatag_action)\
                                   .append(b97_action)\
                                   .append(judgements_action)

In [ ]:
print('  10.e Exporting results to product folder')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה

In [ ]:
SequenceActionsTable.to_csv(r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\SequenceActions.csv', index=False)

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\SequenceActions-Save as DB Table.bat"]);